In [16]:
!curl https://raw.githubusercontent.com/mosesyhc/de300-2025sp-class/refs/heads/main/agnews_clean.csv -O

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 33.2M  100 33.2M    0     0  2068k      0  0:00:16  0:00:16 --:--:-- 3654k 0  2:01:33  0:00:01  2:01:32  47713  0:00:10  0:00:13 2259k


In [19]:
from pyspark.sql import SparkSession

spark = (SparkSession.builder
         .master("local[*]")
         .appName("AG news")
         .getOrCreate()
        )

# agnews = spark.read.csv("agnews_clean.csv", inferSchema=True, header=True)
agnews = spark.read.option("header", True).option("multiLine", False).csv("agnews_clean.csv")


# turning the second column from a string to an array
import pyspark.sql.functions as F
from pyspark.sql.types import ArrayType, StringType
agnews = agnews.withColumn('filtered', F.from_json('filtered', ArrayType(StringType())))

ConnectionRefusedError: [Errno 61] Connection refused

In [13]:
# each row contains the document id and a list of filtered words
agnews.show(5, truncate=30)

+---+------------------------------+
|_c0|                      filtered|
+---+------------------------------+
|  0|[wall, st, bears, claw, bac...|
|  1|[carlyle, looks, toward, co...|
|  2|[oil, economy, cloud, stock...|
|  3|[iraq, halts, oil, exports,...|
|  4|[oil, prices, soar, time, r...|
+---+------------------------------+
only showing top 5 rows



25/05/19 15:06:13 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , filtered
 Schema: _c0, filtered
Expected: _c0 but found: 
CSV file: file:///Users/ljh20/Desktop/DE300/agnews_clean.csv


Task 1: Design the MapReduce functions for calculating the tf-idf measure.

In [14]:
agnews.printSchema()
agnews.show(3, truncate=False)

root
 |-- _c0: string (nullable = true)
 |-- filtered: array (nullable = true)
 |    |-- element: string (containsNull = true)

+---+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|_c0|filtered                                                                                                                                                                                                                                          |
+---+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|0  |[wall, st, bears, claw, back, black, reuters, reuters, short, sellers, wall, street, dwindling, band, ultra, cynics, see

25/05/19 15:06:17 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , filtered
 Schema: _c0, filtered
Expected: _c0 but found: 
CSV file: file:///Users/ljh20/Desktop/DE300/agnews_clean.csv


In [15]:
# flatten csv to (doc id, word) 
def flatten_doc(row):
    doc_id = row['id']
    words = row['filtered']
    return [(doc_id, word) for word in words]

rdd = agnews.rdd.map(lambda row: (row['id'], row['filtered']))
rdd_flattened = rdd.flatMap(flatten_doc)

rdd_flattened.take(5)

25/05/19 15:06:22 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , filtered
 Schema: _c0, filtered
Expected: _c0 but found: 
CSV file: file:///Users/ljh20/Desktop/DE300/agnews_clean.csv
25/05/19 15:06:22 ERROR Executor: Exception in task 0.0 in stage 10.0 (TID 17)
org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/Users/ljh20/opt/anaconda3/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/sql/types.py", line 2378, in __getitem__
    idx = self.__fields__.index(item)
ValueError: 'id' is not in list

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/ljh20/opt/anaconda3/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 1247, in main
    process()
  File "/Users/ljh20/opt/anaconda3/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 1239, in process
    serializer.dump_stream

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 10.0 failed 1 times, most recent failure: Lost task 0.0 in stage 10.0 (TID 17) (dhcp-10-105-43-138.wireless.northwestern.private executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/Users/ljh20/opt/anaconda3/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/sql/types.py", line 2378, in __getitem__
    idx = self.__fields__.index(item)
ValueError: 'id' is not in list

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/ljh20/opt/anaconda3/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 1247, in main
    process()
  File "/Users/ljh20/opt/anaconda3/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 1239, in 

25/05/19 23:37:34 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 127975 ms exceeds timeout 120000 ms
25/05/19 23:37:34 WARN SparkContext: Killing executors is not supported by current scheduler.
25/05/19 23:53:24 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:124)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$

In [ ]:
!curl https://raw.githubusercontent.com/mosesyhc/de300-2025sp-class/refs/heads/main/w.csv -O
!curl https://raw.githubusercontent.com/mosesyhc/de300-2025sp-class/refs/heads/main/bias.csv -O
!curl https://raw.githubusercontent.com/mosesyhc/de300-2025sp-class/refs/heads/main/data_for_svm.csv -O